# Attempt at writing down generalized solver
1. Have written up a library of functions. Testing here to write down a single function to call to get us everything we want. 

In [1]:
from broyden import *
from solver_helper import *

import matplotlib.pyplot as plt 
import pandas as pd
import time

### Goal: 
1. Declare our set of baryons, leptons, and mesons. 
2. In the context of our gm3 model: get all the equations. 
3. Solve. 
4. Print out a data table. 

In [2]:
# have to initialize eos
#gm3 = eos(g_sigma_N = 8.784820, g_omega_N = 8.720086, g_rho_N = 8.544795, g_phi_N = 0.0, b = 0.008628, c = -0.002433,\
#             g_sigma_H = 5.408849, g_omega_H = 5.813391, g_rho_H = 0.0, g_phi_H = -4.110688)
rmf = eos(g_sigma_N = 8.79509376389, g_omega_N = 9.1815177, g_rho_N = 9.7793745, b = 0.00414, c = 0.00716)

In [3]:
# have to declare lists
# note: baryons are capitalized! for reasons... 
baryon_list = [Neutron, Proton]
meson_list = [sigma, omega]
lepton_list = [electron]

In [4]:
# have to initialize baryons with coupling constants
# and sigma meson with sigma meson coupling 
init(rmf, baryon_list, meson_list)

In [15]:
A = sys_eqn_gen(baryon_list, meson_list, lepton_list)
nB = 0.27*0.153*hc**3
subs(A, nB, baryon_list, meson_list, lepton_list);

In [7]:
x_guess = np.array([8.0, 4.50, 210.0, 43.0, 43.0])

In [8]:
ind_vars2 = ind_variables(baryon_list, lepton_list, meson_list)
ind_vars2

[sigma, omega, k_n, k_p, k_e]

In [ ]:
ans2 = Newton(A, ind_vars2, x_guess)

In [17]:
nB = 0.27*0.153*hc**3

baryon_list = [Neutron, Proton]
meson_list = [sigma, omega]
lepton_list = [electron]

x_guess = np.array([8.0, 4.50, 210.0, 43.0, 43.0])

def solve(nB, baryon_list, meson_list, lepton_list, x_guess):
    
    # initialize 
    init(rmf, baryon_list, meson_list)
    
    # generate system of equations and substitute in values 
    A = sys_eqn_gen(baryon_list, meson_list, lepton_list)
    subs(A, nB, baryon_list, meson_list, lepton_list)
    
    # generate list of independent variables
    ind_vars2 = ind_variables(baryon_list, lepton_list, meson_list)
    
    # solve the system 
    return Newton(A, ind_vars2, x_guess)

### Threshold Condition
We have a condition on the baryons. They will begin to occupy the system when their chemical potential satisfies the following equation
$$
    \mu_n - q\mu_e \geq g_{\omega B}\omega + I_{3B}g_{\rho B}\rho + m_* \qquad m_* = m_B - g_\sigma \sigma
$$
which is derived from the the chemical potential of the baryon being greater than or equal to its chemical potential in the case where it's momentum is zero. Ie, 
$$
     {\mu_B}_{\lim k_F \rightarrow 0} = \lim_{k_F \rightarrow 0}\left[\sqrt{k_F^2 + m_*^2} + g_{\omega B}\omega + I_{3B}g_{\rho B}\rho\right] 
$$
At each step, we need to check if the chemical potentials have become significant enough to populate. Adds significant complexity to our system tbh.

## Fresh Attempt:
1. Want to write down a function that will solve everything for us. Ie, we call it, specify the baryon, lepton, and meson lists. And we get back a matrix/Dataframe of the data. Also writes it out to a data file. 

### Establish lists of potential particles that we want to include in our system but would appear at a later time... 

In [19]:
def potential_baryon_gen(baryon_list):
    pot_list = []
    for baryon in baryon_list:
        if (baryon != Proton and baryon != Neutron):
            pot_list.append(baryon)
    return pot_list.sort(key = mass) 

def potential_lepton_gen(lepton_list):
    pot_list = [] 
    for lepton in lepton_list:
        if (lepton != electron):
            pot_list.append(lepton)
    return pot_list 

## Threshold Functions
1. First, need to calculate the chemical potentials for neutron and electron chemical potentials
2. Bare chemical potential of a baryon: gives the chemical potential in the case where the baroyn has zero momentum (analog of chemical potential being just the mass) 
3. Finally declare a threshold function which returns a bool: true or false. 

In [46]:
def neutron_chem_pot_num(fermi, meson_list):
    # gets numerical values for neutron chemical potential 
    bare_chem = np.sqrt(fermi**2 + (Neutron.num_mass - Neutron.num_g_sigma*sigma_field)**2)
    result = 0
    for meson in meson_list:
        if (meson == rho):
            result += Neutron.isospin*Neutron.num_g_rho*rho.num_field
        else:
            result += meson.g_N*meson.num_field
    return bare_chem + result 

In [47]:
def electron_chem_pot_num(fermi):
    # return numerical chemical potential for electron
    return np.sqrt(fermi**2 + electron.num_mass**2)

In [42]:
def bare_chemical_potential(baryon, meson_list):
    # finds bare chemical potential for a baryon given a list of mesons
    # assumes meson objects are filled with field values
    # this code is ugly and is not easily generalizable... 
    bare_chem = 0.0 
    for meson in meson_list:
        if (meson == sigma):
            bare_chem += baryon.num_mass - baryon.num_g_sigma * sigma.num_field 
        elif (meson == omega):
            bare_chem += baryon.num_g_omega * omega.num_field
        elif (meson == rho):
            bare_chem += baryon.num_g_rho * baryon.isospin * rho.num_field 
        elif (meson == phi):
            bare_chem += baroyn.num_g_phi * rho.num_field 
    return bare_chem 

In [48]:
def baryon_threshold(baryon, meson_list):
    # checks to see if combination of neutron and electron chemical potential 
    
    neutron_chem_pot = neutron_chem_pot_num(Neutron.num_kf, meson_list) 
    electron_chem_pot = electron_chem_pot_num(electron.num_kf)
    
    if (neutron_chem_pot - baryon.charge * electron_chem_pot >= bare_chemical_potential(baryon, meson_list)):
        return True
    else:
        return False 
    
def lepton_threshold(lepton):
    # checks to see if electron chemical potential is large enough
    # to support entrance of other leptons, namely, muon 
    if (electron.num_kf >= lepton.mass):
        return True
    else:
        return False 

In [53]:
def frac(fermi, nb):
    return fermi**3 /3/np.pi**2/nb

In [59]:
def full_solve(eos, baryon_list, lepton_list, meson_list):
    # full solver 
    
    # initialize all things... 
    init(eos, baryon_list, meson_list)
    
    # create nB array 
    nB = np.linspace(0.27, 0.50, 0.01) #nB/n0
    nB_mev = nB*n0*hc**3 #nB_mev in mev+3 
    
    # create data array (pre-allocate)
    row_size = len(nB)
    column_size = len(meson_list) + 2*len(baryon_list + lepton_list) + 1 
    data = np.zeros((row_size, column_size), dtype = 'float') 
    data[:,0] = nB 
    
    # create first system: NPE
    current_baryons = [Neutron, Proton]
    current_leptons = [electron] 
    
    # create lists for potential particles 
    potential_baryons = potential_baryon_gen(baryon_list)
    potential_leptons = potential_lepton_gen(lepton_list) 
    

    # independent variables
    ind_vars = ind_variables(current_baryons, current_leptons, meson_list)
    
    # initial guess for NPE matter 
    x_guess = [8.00, 4.50, -2.25, 210.0, 45.0, 45.0] 
    
    
    # iterate through baryon density nB 
    for i in range(len(nB_mev)): 
        
        # update our system: 
        
        Bool = baryon_threshold(baryon, meson_list) 
        if (Bool):
            current_baryons.append(potential_baryons[0])
            potential_baryons.remove(potential_baryons[0])
            x_guess.append('0')
        
        Bool = lepton_threshold(lepton)
        if (Bool):
            current_leptons.append(potential_leptons[0])
            potential_leptons.remove(potential_leptons[0])
            x_guess.append('0')
        
        # 
        init(rmf, baryon_list, meson_list)
            
        # generate system of equations 
        sys_eqn = sys_eqn_gen(current_baryons, meson_list, current_leptons)
        subs(sys_eqn)
        
        # generate independent variables 
        ind_vars = ind_variables(current_baryons, current_leptons, meson_list)
        
        # call solver, broyden returns a vector with solutions to independent variables
        # very importantly, the independent variables are ordered: sigma, omega, rho, phi
        answer = broyden(sys_eqn, ind_vars, x_guess)
        
        # append values to data file 
        for j in range(len(column_size)-1):
            data[i][j+1] = answer[j]
        
        
        # update important values
        pseudo_dict = [ind_vars, answer]
        for k in range(len(ind_vars)):
            if (k == sigma.sym_field):
                sigma.num_field = answer[k]
            elif (k == omega.sym_field):
                omega.num_field = answer[k]
            elif (k == rho.sym_field):
                rho.num_field = answer[k]
            elif (k == phi.sym_field):
                phi.num_field = answer[k]
            elif (k == Neutron.sym_kf):
                Neutron.num_kf = answer[k]
            elif (k == electron.sym_kf):
                electron.num_kf = answer[k]
        
        # update x_guess
        x_guess = answer 
        
    # fill in fractions 
    for rem in range(len(nB)):
        for emilia in range(len(ind_vars)):
            data[rem][emilia + len(meson_list + baryon_list + lepton_list) + 1]\
                = frac(data[rem][0], data[rem][emilia + meson_list + 1])
         
    # convert data array to dataframe 
    data_frame = pd.DataFrame(data, columns = column_name(baryon_list, meson_list, lepton_list))
    
    
    # write out dataframe to csv file 
    data_frame.to_csv('data.csv', float_format = '')
    
    return data_frame 

## $\sigma-\omega$ model attempt

In [2]:
rmf = eos(g_sigma_N = 8.79509376389, g_omega_N = 9.1815177, g_rho_N = 9.7793745, b = 0.00414, c = 0.00716)

In [3]:
baryon_list = [Neutron, Proton]
meson_list = [sigma, omega]
lepton_list = [electron]

In [4]:
full_solve(rmf, baryon_list, meson_list, lepton_list)

AttributeError: 'lepton' object has no attribute 'sym_field'

In [5]:
ind_variables(baryon_list, lepton_list, meson_list)

TypeError: sort() takes no positional arguments

In [7]:
current_baryons = [Neutron, Proton]
current_leptons = [electron]
ind_vars = ind_variables(current_baryons, current_leptons, meson_list)

TypeError: sort() takes no positional arguments

In [13]:
arag = np.array([[1.0,2.0,3.0],[5.0,6.0,7.0]])

In [14]:
A = pd.DataFrame(arag, columns = ['a', 'b', 'c'])

In [18]:
A.to_csv('test.csv', float_format = '{:.4f}'.format)

In [19]:
A

,a,b,c
0,1.0,2.0,3.0
1,5.0,6.0,7.0
